In [1]:
#upload data from csv
import pandas as pd
videosGB = pd.read_csv('OriginalCSVs/GBvideos.csv', sep=',', decimal='.', header=0)
#videosUS = pd.read_csv('OriginalCSVs/USvideos.csv', sep=',', decimal='.', header=0)
#videosCA = pd.read_csv('OriginalCSVs/CAvideos.csv', sep=',', decimal='.', header=0)

#concat videos
videos = pd.concat([videosGB])

#GB: 19000 videos
#GB+US: 39000 videos
#GB+US+CA: 69875 videos
videos.shape

(30237, 16)

In [2]:
#elimina videos duplicados
videos = videos.drop_duplicates('video_id','last',False)
#elimina sem categorias (tags)
nodes = videos[videos.tags != '[none]']
        
#cria csv
newheader = ['id','trending_date','Label','channel_title','category_id','publish_time','tags','views','likes','dislikes','comment_count','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description']
nodes.to_csv('CreatedCSVs/nodesGB.csv',header=newheader,index=False)

#GB: 2000 nos
#GB+US: 5320 nos
#GB+US+CA: 16700 nos
nodes.shape

(2601, 16)

In [4]:
#CORRER SÓ PARA GERAR OS GRUPOS DE CANAIS, COM AS CATEGORIAS ASSOCIADADOS
channel = dict()

for index, node in nodes.iterrows():
    channel_name = node['channel_title']
    if channel.get(channel_name, None) == None:
        channel[channel_name] = set()
    channel[channel_name].add(node['category_id'])
    
#print
len(channel)

for key, value in channel.items():
    print(key, value)

PointlessBlogVlogs {24}
MoreZoella {22}
marshmello {10, 26}
Inthefrow {26}
WWE {17}
The Late Show with Stephen Colbert {24}
SamanthaMariaVlogs {26}
Amelia Liana {26}
Lily Pebbles {26}
Giovannasworld {24}
Estée Lalonde {22}
Hazel Hayes {22}
Jimmy Kimmel Live {24, 10, 23}
The Tonight Show Starring Jimmy Fallon {23}
HOT 97 {10}
Lex Croucher {23}
MadisonBeerMusicVEVO {10}
dodieVEVO {10}
WORLDSTARHIPHOP {10}
MILCK {10}
Grace Helbig {22}
Movieclips Trailers {1}
AT&T {10}
Pop Blast {24}
TBS {24}
The Late Late Show with James Corden {24}
2CELLOS {10}
TheEllenShow {24}
Late Night with Seth Meyers {23}
ddriadon {24}
vlogbrothers {22}
SUPERFRUIT {24}
PsychoSoprano {23}
First We Feast {26}
colinfurze {2}
EinsteinEnt {24}
Shawn Johnson East {22}
DaLordz Prince {1}
Ali-A {20}
LukeBryanVEVO {10}
Sprinkleofglitter {26}
CJENMMUSIC Official {10}
TMZSports {17}
RemyMaVEVO {10}
WIRED {24, 17, 2, 28}
Tu Fan {24}
The Anna Edit {26}
Stephen Howson FC {17}
ncsoft {20}
Tom Fletcher {24}
World of Warcraft {20}


In [3]:
#get tags
nodesTemp = nodes.loc[:, ['video_id','tags','channel_title']]
sources = []
targets = []

#auxiliar vectors that save the indexes to access to the nodes quickly
sourcesAux = []
targetsAux = []

#criar vetor de tags
for index, tags in nodesTemp.iterrows():
    tags['tags'] = tags['tags'].split('"|"')  
    #compor a ultima tag
    size = len(tags['tags'])
    last = tags['tags'][size-1]
    tags['tags'][size-1] = last.split('"')[0]
    #compor a primeira tag
    tag1 = tags['tags'][0].split('|"')
    tags['tags'][0] = tag1[0]
    if len(tag1) == 2:
        tags['tags'].append(tag1[1])
    
        
print("done") 
nodesTemp.shape

done


(1986, 3)

In [8]:
#CORRER SÓ PARA GERAR OS GRUPOS DE CANAIS 
channel_conn = dict()

def comparelists( c1, list1, c2, list2 ):
    for x1 in list1:
        for x2 in list2:
            if x1 == x2:
                channel_conn[c1].add(c2)
                return

for i1, video1 in nodesTemp.iterrows(): 
    if  channel_conn.get(video1['channel_title'], None) == None:
        channel_conn[video1['channel_title']] = set()
    for i2, video2 in nodesTemp.iterrows(): 
        if i2 > i1:             
            #se ainda nao estiver na lista desse video
            connections = channel_conn.get(video1['channel_title'], None)
            if connections == None or video2['channel_title'] not in connections:
                #o video1 nao esta na lista do video2
                connections2 = channel_conn.get(video2['channel_title'], None)
                if connections2 == None or video1['channel_title'] not in connections2:
                    #verificar se existe alguma tag igual
                    comparelists(video1['channel_title'],video1['tags'],video2['channel_title'],video2['tags'])
                
print("done")

done


In [1]:
#CORRER SÓ PARA GERAR OS GRUPOS DE CANAIS 

len(channel_conn)

import sys
sys.stdout = open('channels.txt','wt')

for key, value in channel_conn.items():
    print(key, value)
    print("/n")

NameError: name 'channel_conn' is not defined

In [4]:
#CORRER SO PARA GERAR UM GRAFO COMPLETO (com edges positivos e negativos)
sourcesC = []
targetsC = []
labelsC = []

halfNodes = nodesTemp.iloc[:1000,:]
#halfNodes = nodesTemp

def labeling(list1, list2 ):
    for x1 in list1:
        for x2 in list2:
            if x1 == x2:
                labelsC.append('+')
                return
    labelsC.append('-')

#criar lista de edges (todos os nós para todos os nós)
for i1, node1 in halfNodes.iterrows(): 
    for i2, node2 in halfNodes.iterrows(): 
        if i2 > i1:
            sourcesC.append(node1['video_id'])
            targetsC.append(node2['video_id'])
            labeling(node1['tags'],node2['tags'])

#ver se para cada par existe uma tag em comum, se sim adiciona como positivo, se nao, e negativo

#criar dataframe especial         
df = {'Source' : sourcesC, 'Target' : targetsC, 'PositiveNegative' : labelsC}   
edges = pd.DataFrame(df)
edges['Type'] = 'Undirected'
edges.to_csv('CreatedCSVs/edgesHalfCompleteGB.csv',index=False)

#GB: 1.971.105 edges 
#GB/2: 499.500
edges.shape

(499500, 4)

In [5]:
def comparelists( id1, list1, index1, id2, list2, index2 ):
    for x1 in list1:
        for x2 in list2:
            if x1 == x2:
                sources.append(id1)
                sourcesAux.append(index1)
                targets.append(id2)
                targetsAux.append(index2)
                return
         
#para cada node vamos procurar todos aqueles que tem a mesma tag    
for i1, tags1 in nodesTemp.iterrows(): 
    for i2, tags2 in nodesTemp.iterrows(): 
        #compara com os seguintes se existir tags
        if i2 > i1: 
            #verificar se existe alguma tag igual
            comparelists(tags1['video_id'],tags1['tags'],i1,tags2['video_id'],tags2['tags'],i2)

print("done")

KeyboardInterrupt: 

In [12]:
#create dataframe            
df = {'Source' : sources, 'Target' : targets}   
edges = pd.DataFrame(df)
edges['Type'] = 'Undirected'
edges.to_csv('CreatedCSVs/edgesUSCAGB.csv',index=False)

#create auxiliary dataframe
dfAux = {'Source' : sourcesAux, 'Target' : targetsAux}   
edgesAux = pd.DataFrame(dfAux)

#GB: 86000 edges
#GB+US: 574000 edges
#GB+US+CA: 
edges.shape

(574128, 3)

In [13]:
#criar edges com pesos de acordo com o numero de tags em comum
weights = []
def getweights( ite, list1, list2 ):
    w = 0
    for x1 in list1:
        for x2 in list2:
            if x1 == x2:
                w += 1
                break
    weights.append(w)

    
#source and target iterator
for e1, edge in edgesAux.iterrows(): 
    list1 = nodesTemp['tags'][edge['Source']]
    list2 = nodesTemp['tags'][edge['Target']]
    getweights(e1, list1, list2)
    
print("done")

done


In [14]:
edgesWeight = edges
edgesWeight['Weight'] = weights
edgesWeight.to_csv('CreatedCSVs/edgesWeightnedUSCAGB.csv',index=False)

#remover todos os edges com peso = 1
#GB: 24000 edges
#GB+US: 351498 edges 

#edgesWeight1 = edgesWeight[edgesWeight.Weight != 1]
#edgesWeight1.to_csv('edgesWeightned1.csv',index=False)
#edgesWeight1.shape

#se retirar todos os edges com peso 1 fico com 94500 edges e 3000 nós (GB+US)

(351498, 4)